<a href="https://colab.research.google.com/github/dlguswn3659/2020_Image_Lab/blob/master/hair_removal/hair_removal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive

drive.mount('/content/gdrive')

In [ ]:
!pip install -q pydicom

In [ ]:
from PIL import Image
import cv2
import os
import pandas as pd
import matplotlib.pyplot as plt
import pydicom
import matplotlib.pyplot as plt
from pydicom.data import get_testdata_files

In [ ]:
from google.colab.patches import cv2_imshow

!curl -o logo.png https://colab.research.google.com/img/colab_favicon_256px.png
import cv2

In [ ]:
# -*- coding: utf-8 -*-
"""
Following are the DHR tasks followed in this example code:
    
    -- Applying Morphological Black-Hat transformation
    -- Creating the mask for InPainting task
    -- Applying inpainting algorithm on the image
"""
# define location of dataset
folder = '/content/gdrive/My Drive/ISIC-melanoma/train/'
new_storage_folder = '/content/gdrive/My Drive/hair-removal-image/'

start_num = 10000
end_num = 70000

file_startname = 'ISIC_00'

if start_num >= 100000:
  file_startname = 'ISIC_0'
elif start_num >= 1000000:
  file_startname = 'ISIC_'

for i in range (start_num, end_num):
  if os.path.isfile(folder + file_startname + str(i) + '.dcm'):
    filename = folder + file_startname + str(i) + '.dcm'
    dataset = pydicom.dcmread(filename)

    # print( src.shape )
    # cv2.imshow("original Image" , src )
    if 'PixelData' in dataset:
        rows = int(dataset.Rows)
        cols = int(dataset.Columns)
        print("Image size.......: {rows:d} x {cols:d}, {size:d} bytes".format(
            rows=rows, cols=cols, size=len(dataset.PixelData)))
        if 'PixelSpacing' in dataset:
            print("Pixel spacing....:", dataset.PixelSpacing)

    # use .get() if not sure the item exists, and want a default value if missing
    print("Slice location...:", dataset.get('SliceLocation', "(missing)"))
    print("file name : ", file_startname + str(i))

    # src = cv2.cvtColor(dataset.pixel_array, cv2.COLOR_RGB2BGR)
    src = dataset.pixel_array

    # plt.imshow(src, cmap='gray')
    # plt.imshow(src)
    # plt.show()

    print( src.shape )
    cv2_imshow(src )

    # Convert the original image to grayscale
    grayScale = cv2.cvtColor(dataset.pixel_array, cv2.COLOR_BGR2GRAY )
    print("GrayScaled Image")
    cv2_imshow(grayScale)
    # plt.imshow(grayScale)
    # plt.show()
    cv2.imwrite(file_startname + str(i) + 'grayScale.jpg', grayScale, [int(cv2.IMWRITE_JPEG_QUALITY), 90])

    # Kernel for the morphological filtering
    kernel = cv2.getStructuringElement(1,(17,17))

    # Perform the blackHat filtering on the grayscale image to find the 
    # hair countours
    blackhat = cv2.morphologyEx(grayScale, cv2.MORPH_BLACKHAT, kernel)
    print("BlackHat filtering result")
    cv2_imshow(blackhat)
    # plt.imshow(blackhat)
    # plt.show()
    cv2.imwrite(file_startname + str(i) + 'blackhat.jpg', blackhat, [int(cv2.IMWRITE_JPEG_QUALITY), 90])

    # intensify the hair countours in preparation for the inpainting 
    # algorithm
    ret,thresh2 = cv2.threshold(blackhat,10,255,cv2.THRESH_BINARY)
    print("Thresholded image for inpainting")
    print( thresh2.shape )
    cv2_imshow(thresh2)
    # plt.imshow(thresh2)
    # plt.show()
    cv2.imwrite(file_startname + str(i) + 'thresholded.jpg', thresh2, [int(cv2.IMWRITE_JPEG_QUALITY), 90])

    # inpaint the original image depending on the mask
    dst = cv2.inpaint(src,thresh2,1,cv2.INPAINT_TELEA)
    print("Image after Inpainting")
    cv2_imshow(dst)
    # plt.imshow(dst)
    # plt.show()
    cv2.imwrite(new_storage_folder + file_startname + str(i) + 'Inpainted.jpg', dst, [int(cv2.IMWRITE_JPEG_QUALITY), 90])